In [ ]:
import uuid
from concurrent.futures import ThreadPoolExecutor
from functools import cache
from subprocess import run

from tomli import load
from tqdm import tqdm


def get_deps(package):
    pixi_toml = f"/home/shh/projects/holoviz/repos/{package}/pixi.toml"

    with open(pixi_toml, "rb") as f:
        data = load(f)

    ignore_features = ["test-gpu", "build", "lint", "lite", "test-ui", "type"]
    features = [
        list(v.get("dependencies", []))
        for k, v in data["feature"].items()
        if k not in ignore_features
    ]
    deps = {*data["dependencies"], *sum(features, [])}  # noqa: RUF017
    return deps


@cache
def func(*deps):
    cmd = [
        "mamba",
        "create",
        "-n",
        uuid.uuid4().hex,
        "python=3.13",
        *deps,
        "-c",
        "conda-forge/label/python_rc",
        "-c",
        "pyviz/label/dev",
        "--dry-run",
    ]
    if deps:
        cmd.append("--offline")
    output = run(cmd, capture_output=True)
    return output


def check(deps):
    with ThreadPoolExecutor() as ex:
        futures = list(tqdm(ex.map(func, deps), total=len(deps)))
    for dep, future in zip(deps, futures):
        if future.returncode:
            print(dep, "failed", flush=True)
        else:
            print(dep, "success", flush=True)


print_output = lambda dep: print(func(dep).stdout.decode())
no_check = func()

In [ ]:
check(get_deps("panel"))

In [ ]:
deps = [
    "fastparquet",
    "numba",
    "pyarrow",
    "vtk",
    "scikit-image",
    "python-duckdb",
    "pywin32",  # jupyter ecosystem
]

check(deps)